In [1]:
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, LambdaCallback
from keras.callbacks import EarlyStopping, TensorBoard
import argparse
import midi
import os

from constants import *
from dataset import *
from generate import *
from midi_util import midi_encode
from model import *
from keras import backend as K

import gc
gc.collect()
from keras import backend as K
K.clear_session()

/home/egor/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
styles[0]

['data/Bach1']

In [3]:
print('Loading data')
train_data, train_labels = load_all(styles, BATCH_SIZE, SEQ_LEN)

Loading data


In [4]:
for i in range(3):
    print(train_data[i].shape)

(340, 128, 48, 3)
(340, 128, 48, 3)
(340, 128, 16)


In [5]:
(train_data[2][4]==0).any()

True

In [6]:
models = build_models(time_steps=SEQ_LEN, input_dropout=0.2, dropout=0.5)

In [7]:
cbs = [
    ModelCheckpoint(MODEL_FILE, monitor='loss', save_best_only=True, save_weights_only=True),
    EarlyStopping(monitor='loss', patience=5),
    #TensorBoard(log_dir='out/logs', histogram_freq=1)
]

print('Training')
models[0].fit(train_data, train_labels, validation_split=0.05,
              epochs=1, callbacks=cbs, batch_size=1)


Training
Train on 323 samples, validate on 17 samples
Epoch 1/1
  8/323 [..............................] - ETA: 15:58 - loss: 0.4155

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-63acfe02e6db>", line 9, in <module>
    epochs=1, callbacks=cbs, batch_size=1)
  File "/home/egor/.local/lib/python3.5/site-packages/keras/engine/training.py", line 1705, in fit
    validation_steps=validation_steps)
  File "/home/egor/.local/lib/python3.5/site-packages/keras/engine/training.py", line 1235, in _fit_loop
    outs = f(ins_batch)
  File "/home/egor/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py", line 2478, in __call__
    **self.session_kwargs)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py", line 789, in run
    run_metadata_ptr)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py", line 997, in _run
    feed_dict_string, options, run_metadata)
  File "/usr/local/li

KeyboardInterrupt: 

In [24]:
models = build_models()
models[0].load_weights(os.path.join(OUT_DIR, 'model.h5'))
write_file('output/bach3', generate(models, 8))

  0%|          | 0/128 [00:00<?, ?it/s]

Generating with no styles:


100%|██████████| 128/128 [01:44<00:00,  1.22it/s]

Writing file out/samples/output/bach3_0.mid


In [9]:
fpath = os.path.join(SAMPLES_DIR, 'encode_decoded_song' + '_' + str(i) + '.mid')
pattern = midi.read_midifile('data/Bach1/Toccata & Fuga in F-Dur, BWV 540.mid')
result = midi_decode(pattern)
mf = midi_encode(unclamp_midi(clamp_midi(result)))
midi.write_midifile(fpath, mf)

In [23]:
# A = np.array([[np.ones((2,3)), np.eye(3)[:2]]])
# A = K.variable(A)
# print(K.eval(A))
# print(A.shape)
# # x = Dense(4, kernel_initializer='Ones')(A)\
# print('slice x', K.eval(A[:, :, :-1, :]))

# x = Lambda(lambda x: tf.pad(x[:, :, :-1, :], 
#                         [[0, 0], [0, 0], [1, 0], [0, 0]]))(A)

# print(x.shape)
# print('x', K.eval(x))


In [24]:
# models[0].save_weights(os.path.join(OUT_DIR, 'raw_model.h5'))

In [19]:
# models = build_models()
# models[0].load_weights(MODEL_FILE)
# write_file('output2', generate(models, 4, styles))

In [15]:
compute_beat(3, 16)

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [141]:
SEQ_LEN

128

In [169]:
d = deque([np.zeros((2, 3)) for _ in range(2)], maxlen=2)

In [170]:
d

deque([array([[0., 0., 0.],
              [0., 0., 0.]]), array([[0., 0., 0.],
              [0., 0., 0.]])])

In [173]:
d.append(np.ones((2,3)))

In [174]:
d

deque([array([[1., 1., 1.],
              [1., 1., 1.]]), array([[1., 1., 1.],
              [1., 1., 1.]])])

In [142]:
NOTES_PER_BAR

16

In [191]:
# for i, result in enumerate(generate(models, 4, styles)):
# #     print(i)
# #     print(np.array(result).shape)
# #     print(unclamp_midi(result).shape)
# #     print(midi_encode(unclamp_midi(result)))
#     break

In [62]:
g = MusicGeneration(styles[0])
a = g.build_time_inputs()

In [63]:
type(a)

tuple

In [19]:
for i in a:
    print(i.shape)

(128, 48, 3)
(128, 16)
(128, 1)


In [21]:
(a[0]==0).any()

True

In [29]:
(a[1]==0).any()

True

In [65]:
for i in process_inputs([a]):
    print(i.shape)

(1, 128, 48, 3)
(1, 128, 16)
(1, 128, 1)


In [57]:
models[1].predict(process_inputs([a]))[:, -1:, :].shape

(1, 1, 48, 256)

In [74]:
note_features = models[1].predict(process_inputs([a]))[:, -1:, :]
note_features[0, : ,: , :].shape

(1, 48, 256)

In [76]:
b = g.build_note_inputs(note_features[0, : ,: , :])
for i in b:
    print(i.shape)

(1, 48, 256)
(1, 48, 3)
(1, 1)


In [80]:
# b

In [77]:
for i in process_inputs([b]):
    print(i.shape)

(1, 1, 48, 256)
(1, 1, 48, 3)
(1, 1, 1)


In [85]:
pr = np.array(models[2].predict(process_inputs([b])))
pr.shape

(1, 1, 48, 3)

In [94]:
pr2 = pr[0][-1]

In [96]:
pr2[2, -1]

-0.0836291

In [98]:
prob = apply_temperature(pr2[2, :-1], g.temperature)

In [99]:
prob

array([0.49578953, 0.4675863 ], dtype=float32)

In [89]:
g.choose(pr[0][-1], 0)